In [1]:

from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Ready_for_analysis').getOrCreate()



In [2]:

#IMPORTING SETUP PACKAGES
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)



In [3]:
number_of_top_items = '5000'
plot_name = number_of_top_items[0] + 'k'

#print(plot_name)

df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top'+number_of_top_items+'_seller_YTD_group_by_item_visit_20210226.pickle')


In [4]:
#print(df.item_code.nunique())


df['log_avg_unique_views']= np.log(df['avg_unique_views'])

In [5]:
df.head()

,min_date,max_date,item_code,item_price,delivery_weeks,avg_sales,avg_unique_views,conv_rate,bin,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales,log_avg_unique_views,log_conv_rate
0,2020-01-01,2020-05-23,000000001000012530,99.99,1,0.520833,35.743056,0.014573,144,000000008000000774,ESSZIMMERSTÜHLE&BÄNKE,BÄNKE,BÄNKE,4.605071,0.000100,-0.652133,3.576356,-4.221842
1,2020-01-01,2020-01-02,000000001000012794,219.99,2,0.000000,27.000000,0.000001,2,000000008000000826,BETTEN,BETTGESTELLE,MASSIVHOLZBETTEN,5.393583,0.693197,-9.210340,3.295837,-9.210340
2,2020-01-01,2020-03-13,000000001000021922,129.99,2,1.589041,147.191781,0.010797,73,000000001000021922-P,BAD,BADSCHRÄNKE,WASCHBECKENUNTERSCHRÄNKE,4.867458,0.693197,0.463194,4.991736,-4.519385
3,2020-01-01,2020-01-01,000000001000022179,549.99,3,0.000000,50.000000,0.000001,1,000000008000001920,BETTEN,BETTGESTELLE,POLSTERBETTEN,6.309900,1.098646,-9.210340,3.912023,-9.210340
4,2020-01-01,2020-01-01,000000001000024320,99.99,1,0.000000,8.000000,0.000001,1,000000008000002133,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.605071,0.000100,-9.210340,2.079442,-9.210340


In [6]:
print(df.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg_sales                float64
avg_unique_views         float64
conv_rate                float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
log_avg_unique_views     float64
log_conv_rate            float64
dtype: object


In [7]:
def filter_df(DF = df,bins=6, avg_PDP = 19.):
    df_filt = DF[DF['bin'] > bins]
    df_items = df_filt.groupby('item_code').item_price.nunique() > 1
    df_filt  = df_filt.merge(df_items,on='item_code',how='inner')
    df_filt  = df_filt[df_filt['item_price_y']== True]
    #filter PDP
    #print(df_filt)
    df_filt = df_filt[df_filt['avg_unique_views'] > avg_PDP]
    #print(df_filt)
    return df_filt

In [8]:
df_filt = filter_df(df)


print(df_filt.item_code.nunique())


4181


In [9]:
#introducing the similarity score
sim_score = pd.read_parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/recommendations_prod/part-00000-5f77392a-0bfa-4f0b-a4ae-d816ce2d71de-c000.snappy.parquet')

In [10]:
sim_score

,sku,recommended_sku,score
0,000000001000200072,000000001000200087,0.260000
1,000000001000200072,000000001000183538,0.216580
2,000000001000200072,000000001000142209,0.200000
3,000000001000200072,000000001000204906,0.190794
4,000000001000200072,000000001000183468,0.183980
...,...,...,...
3827600,000000001000064787,000000001000115010,0.203238
3827601,000000001000064787,000000001000112641,0.201339
3827602,000000001000064787,000000001000046204,0.200052
3827603,000000001000064787,000000001000114086,0.199867


In [11]:
# Filter table with only topX item_code

unique_item_codes = list(df_filt.item_code.unique())


sim_score_filt = sim_score[sim_score['sku'].isin(unique_item_codes)]

sim_score_filt

,sku,recommended_sku,score
780,000000001000135008,000000001000135012,0.881024
781,000000001000135008,000000001000121202,0.715546
782,000000001000135008,000000001000167468,0.633481
783,000000001000135008,000000001000166799,0.564164
784,000000001000135008,000000001000208980,0.544883
...,...,...,...
3827360,000000001000176744,000000001000171107,0.339302
3827361,000000001000176744,000000001000163202,0.335340
3827362,000000001000176744,000000001000163190,0.327887
3827363,000000001000176744,000000001000176438,0.324293


In [12]:
#Create a map with X subset for X linear fit:

def create_map(list_of_items = unique_item_codes , df=sim_score_filt, score_cut = 0.):
    map_of_items = {}
    for item_code in list_of_items:
        df_mod = df[df['sku'] == item_code]
        df_score_filt = df_mod[df_mod['score'] > score_cut]
        final_set_sku = list(df_score_filt.recommended_sku)
        final_set_sku.insert(0,item_code)
        map_of_items[item_code] = final_set_sku
    #print(map_of_items)
    return map_of_items


In [13]:
sim_score_bin = [0.16]#0.3,0.4,0.5,0.6,0.7,0.8,0.9]




In [14]:
map_of_map_of_items = {}
for sim_score_i in sim_score_bin:
    Map_of_Items = create_map(score_cut = sim_score_i)
    map_of_map_of_items[str(sim_score_i)] = Map_of_Items
#print(map_of_map_of_items)

In [15]:
#load greater dataset
largest_dataset = '/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/All_catalogue_seller_YTD_group_by_item_visit_20210226.pickle'

#df_k = pd.read_pickle(dataset10k)
df_k = pd.read_pickle(largest_dataset)

In [16]:
PDP_bin = [19]#,30,40,50]

In [17]:
df_PDP_bin_filt = {}
for PDP_view in PDP_bin:
    df_k_filt = filter_df(df_k, avg_PDP= PDP_view)
    df_PDP_bin_filt[PDP_view] = df_k_filt
    #print(df_k_filt)

#print(df_PDP_bin_filt)


In [18]:
def create_map_of_dataframes(items_map = Map_of_Items,df=df_filt):
    map_of_df = {}
    for key in items_map.keys():
        items  = items_map[key]
        #print(f'Item_code = {key}')

        df_mod = df[df['item_code'].isin(items)]
        #
        #print(f'Number of items per cluster {df_mod.item_code.nunique()}')
        map_of_df[key]= df_mod
    return map_of_df



In [19]:
import numpy as np
def run_model(item_code, df, map_results):
    #print('\n')
    wls_cr_with_fe = smf.wls("log_avg_unique_views ~ log_price + log_delivery_weeks + C(item_code)", df, weights=df['bin']).fit()
    wls_cr_with_fe.summary()
    beta_log_price, beta_std_err_log_price = wls_cr_with_fe.params[-2].round(4),wls_cr_with_fe.bse[-2].round(4)
    beta_log_del, beta_std_err_log_del = wls_cr_with_fe.params[-1].round(4),wls_cr_with_fe.bse[-1].round(4)
    Result_log_price = 'Log_price  = ' + str(beta_log_price) + ' ± ' + str(beta_std_err_log_price)
    Result_log_del = 'Log_del_week = ' + str(beta_log_del) + ' ± ' + str(beta_std_err_log_del)
    flag_stat_ok = False
    if beta_log_price != 0 :
        flag_stat_ok = np.fabs((3*beta_std_err_log_price)/beta_log_price) < 1.
    #flag_stat_ok = fabs(beta_std_err_log_price/beta_log_price)

    if flag_stat_ok:
        #print(f'Enough statistics for the item = {item_code}')
        #print(Result_log_price)
        #print(Result_log_del)
        map_results[item_code] = [beta_log_price, beta_std_err_log_price, beta_log_del,beta_std_err_log_del, df.item_code.nunique(), True]
        #print(map_results[item_code])
        return True
    else:
        #print(f'Not enough statistics for the item = {item_code}')
        #print(Result_log_price)
        #print(Result_log_del)
        map_results[item_code] = [beta_log_price, beta_std_err_log_price, beta_log_del,beta_std_err_log_del, df.item_code.nunique(), False]
        #print(map_results[item_code])
        return False


In [20]:
df_result = pd.DataFrame()

map_of_maps_of_df = {}
#print('Cut on PDP, sim_score, result = ')
for PDP_cut in df_PDP_bin_filt.keys():
    for score_cut in  map_of_map_of_items.keys():
        map_of_dfs = create_map_of_dataframes(map_of_map_of_items[score_cut],df=df_PDP_bin_filt[PDP_cut])
        map_of_maps_of_df[PDP_cut] = map_of_dfs
        print(f'Number of items passing the PE cut {len(map_of_dfs)}')
        map_result = {}
        #print("Model used: conv_rate ~ log_price + log_delivery_weeks + C(item_code)\n")
        map_items_stat = {}
        for key in map_of_dfs.keys():
            #map_of_dfs[key].head()
            map_items_stat[key] = run_model(key, map_of_dfs[key], map_result)
            #print(map_result)
        items_stat_series = pd.Series(map_items_stat)
        #print(map_items_stat)
        #print(items_stat_series)
        x,y, y_err, rel_err = {'True':[],'False':[]},{'True':[],'False':[]},{'True':[],'False':[]},{'True':[],'False':[]}
        for i_code,flag in items_stat_series.items():
        #flag=T
            if flag == True:
                y['True'].append(map_result[i_code][0])
                y_err['True'].append(map_result[i_code][1])
                x['True'].append(map_result[i_code][-2])
                #print('True',map_result[i_code][0],map_result[i_code][1])
                rel_err['True'].append(abs(map_result[i_code][1]/map_result[i_code][0]))

            else:
                y['False'].append(map_result[i_code][0])
                y_err['False'].append(map_result[i_code][1])
                x['False'].append(map_result[i_code][-2])
                #print('False',map_result[i_code][0],map_result[i_code][1])
                rel_err['False'].append(abs(map_result[i_code][1]/map_result[i_code][0]))

                continue
        #print(f'{PDP_cut},{score_cut} , {len(x)}')
        df_result = pd.DataFrame.from_dict(map_result, orient='index', columns = ['View_PE', 'View_PE_err', 'log_del_week', 'log_del_week_err', 'Cluster_size', 'has_enough_stat'])
        print (df_result.head(100))

Number of items passing the PE cut 4181
                    View_PE  View_PE_err  log_del_week  log_del_week_err  \
000000001000012530  -1.6265       0.4565        0.0928            0.0298   
000000001000021922  -1.0147       0.1548       -0.0429            0.0294   
000000001000034762  -1.3782       0.3497       -0.0082            0.0724   
000000001000125896  -1.6598       0.3366       -0.0819            0.0510   
000000001000199717  -0.1353       0.6411        0.1802            0.0459   
000000001000005727  -0.9853       0.2722        0.1099            0.0321   
000000001000007327  -0.3818       0.4126        0.0850            0.0641   
000000001000007822  -2.5172       0.8027        0.1204            0.0611   
000000001000016194  -0.6809       0.7536        0.0858            0.0695   
000000001000022565  -4.4744       1.5870       -0.2397            0.2133   
000000001000050787  -2.9477       0.6452       -0.3324            0.1266   
000000001000116614  -1.0357       0.3534       -

/Users/gabriele.sabato/anaconda3/envs/pythonProject/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/gabriele.sabato/anaconda3/envs/pythonProject/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/gabriele.sabato/anaconda3/envs/pythonProject/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/gabriele.sabato/anaconda3/envs/pythonProject/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with l

In [21]:
len(df_result.index)

4181

In [24]:
df_result_true = df_result[df_result['has_enough_stat']==True]

In [26]:
print(df_result_true['View_PE'].max(),df_result_true['View_PE'].min())

718128115022.6685 -14.9175


In [28]:
print(df_result_true['View_PE'].nlargest(5),'\n',df_result_true['View_PE'].nsmallest(5) )

000000001000189701    7.181281e+11
000000001000166901    7.231200e+00
000000001000166893    5.953300e+00
000000001000166811    3.761100e+00
000000001000100147    3.760000e+00
Name: View_PE, dtype: float64 
 000000001000043212   -14.9175
000000001000123336    -9.8241
000000001000079660    -9.0082
000000001000065870    -8.1032
000000001000206918    -7.6650
Name: View_PE, dtype: float64


In [31]:
df_result_true = df_result_true[(df_result_true['View_PE'] < 10.) & (df_result_true['View_PE'] > -10.)]

In [32]:
print(df_result_true['View_PE'].mean())

-1.8787522714203566


In [35]:
import math

In [36]:
print(df_result_true['View_PE'].std()/math.sqrt(len(df_result_true.index)))

0.024867347691627965


In [38]:
#df_result_true[df_result_true['View_PE'].isna()]

,View_PE,View_PE_err,log_del_week,log_del_week_err,Cluster_size,has_enough_stat


In [40]:
df_result_true_final = df_result_true[['View_PE','View_PE_err']]

In [41]:
df_result_true_final.head()


,View_PE,View_PE_err
000000001000012530,-1.6265,0.4565
000000001000021922,-1.0147,0.1548
000000001000034762,-1.3782,0.3497
000000001000125896,-1.6598,0.3366
000000001000005727,-0.9853,0.2722


In [42]:
df_result_true_final.to_csv('./../raw_data/Excel_files/Top5k_View_PE.csv')
